# Milvus

이 노트북에서는 Milvus 벡터스토어를 시작하는 방법을 다룹니다.

Milvus는 클라우드 네이티브 벡터 데이터베이스로, AI 애플리케이션을 위한 대규모 벡터 검색을 지원합니다. 확장성과 성능에 중점을 둔 오픈소스 프로젝트로, 수십억 개의 벡터를 처리할 수 있습니다.

**주요 특징**
- 클라우드 네이티브 아키텍처
- 수십억 개의 벡터 처리 가능
- 다양한 벡터 인덱스 알고리즘 지원
- 실시간 검색 및 업데이트
- 수평적 확장성
- 다양한 유사도 메트릭 지원
- 하이브리드 검색 (벡터 + 스칼라)

**참고링크**

- [Milvus LangChain 문서](https://python.langchain.com/docs/integrations/vectorstores/milvus/)
- [Milvus 공식문서](https://milvus.io/docs)
- [LangChain 지원 VectorStore 리스트](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)
- [Zilliz Cloud (Managed Milvus)](https://cloud.zilliz.com/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
try:
    from langchain_teddynote import logging
    # 프로젝트 이름을 입력합니다.
    logging.langsmith("CH09-VectorStores-Milvus")
    print("✅ LangSmith with teddynote 설정 완료")
except ImportError:
    print("⚠️ langchain-teddynote 패키지를 찾을 수 없습니다.")
    print("다음 명령으로 설치하세요: uv add langchain-teddynote")
    
    # 대안: 표준 LangSmith 설정
    import os
    # os.environ["LANGCHAIN_TRACING_V2"] = "true"
    # os.environ["LANGCHAIN_PROJECT"] = "CH09-VectorStores-Milvus"
    print("✅ 표준 LangSmith 설정으로 대체")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH09-VectorStores-Milvus
✅ LangSmith with teddynote 설정 완료


## 패키지 설치

Milvus 사용에 필요한 패키지를 설치합니다.

In [3]:
# 필요한 패키지 설치
# !pip install pymilvus langchain-milvus
print("필요한 패키지가 설치되어 있는지 확인하세요:")
print("- pymilvus")
print("- langchain-milvus")
print("- langchain-openai")
print("- langchain-community")
print("\n설치 명령:")
print("pip install pymilvus langchain-milvus langchain-openai langchain-community")

필요한 패키지가 설치되어 있는지 확인하세요:
- pymilvus
- langchain-milvus
- langchain-openai
- langchain-community

설치 명령:
pip install pymilvus langchain-milvus langchain-openai langchain-community


## 샘플 데이터셋 로드

샘플 데이터셋을 로드합니다.

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_milvus import Milvus
import os

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/nlp-keywords.txt")
loader2 = TextLoader("data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
print(f"NLP 문서 수: {len(split_doc1)}")
print(f"Finance 문서 수: {len(split_doc2)}")
print(f"총 문서 수: {len(split_doc1) + len(split_doc2)}")

NLP 문서 수: 11
Finance 문서 수: 6
총 문서 수: 17


## VectorStore 생성

Milvus 벡터스토어를 생성합니다. Milvus Lite (인메모리), 로컬 Docker 인스턴스, Zilliz Cloud 세 가지 방법을 제공합니다.

### 1. Milvus Lite 설정 (인메모리)

Milvus Lite는 경량화된 버전으로 별도 서버 없이 사용할 수 있습니다.

In [6]:
def create_milvus_lite_connection():
    """
    Milvus Lite (인메모리)로 연결하는 함수
    
    Returns:
        Milvus: Milvus 벡터스토어 또는 None
    
    Usage:
        # 별도 설치 없이 바로 사용 가능
        milvus_db = create_milvus_lite_connection()
        if milvus_db:
            print("✅ 연결 성공!")
        else:
            print("❌ 연결 실패")
    """
    try:
        print("🔄 Milvus Lite (인메모리) 연결 시도...")
        
        # Milvus Lite 연결 (":memory:" 사용)
        milvus_db = Milvus.from_documents(
            documents=split_doc1,
            embedding=OpenAIEmbeddings(),
            connection_args={
                "uri": ":memory:",  # 인메모리 모드
            },
            collection_name="lite_documents",
            drop_old=True  # 기존 컬렉션 삭제
        )
        
        print("✅ Milvus Lite 벡터 저장소 생성 완료")
        return milvus_db
        
    except Exception as e:
        print(f"❌ Milvus Lite 연결 실패: {e}")
        print(f"💡 해결 방법:")
        print(f"   1. pymilvus 패키지가 설치되어 있는지 확인")
        print(f"   2. pip install pymilvus langchain-milvus")
        return None

# 함수 실행 및 테스트
milvus_lite_db = create_milvus_lite_connection()

# 연결 상태 확인
if milvus_lite_db is not None:
    print("\n🎯 Milvus Lite 연결 정보:")
    print(f"   • 모드: 인메모리 (Milvus Lite)")
    print(f"   • 컬렉션명: lite_documents")
    print(f"   • 연결 상태: 활성화")
    print(f"   • 특징: 별도 서버 불필요, 개발/테스트용 최적")
else:
    print("\n⚠️ Milvus Lite 연결에 실패했습니다.")

2025-10-17 16:31:40,484 [ERROR][create_connection]: Failed to create new connection using: :memory: (_utils.py:48)


🔄 Milvus Lite (인메모리) 연결 시도...
❌ Milvus Lite 연결 실패: <ConnectionConfigException: (code=1, message=uri: :memory: is illegal, needs start with [unix, http, https, tcp] or a local file endswith [.db])>
💡 해결 방법:
   1. pymilvus 패키지가 설치되어 있는지 확인
   2. pip install pymilvus langchain-milvus

⚠️ Milvus Lite 연결에 실패했습니다.


### 2. 로컬 Milvus 설정 (Docker)

Docker를 사용하여 로컬에서 완전한 Milvus 서버를 실행할 수 있습니다.

In [ ]:
# Docker Compose 파일 생성 (포트 9091:9091, 19530:19530)
docker_compose_content = """version: '3.5'

services:
  etcd:
    container_name: milvus-etcd
    image: quay.io/coreos/etcd:v3.5.5
    environment:
      - ETCD_AUTO_COMPACTION_MODE=revision
      - ETCD_AUTO_COMPACTION_RETENTION=1000
      - ETCD_QUOTA_BACKEND_BYTES=4294967296
      - ETCD_SNAPSHOT_COUNT=50000
    volumes:
      - ${DOCKER_VOLUME_DIRECTORY:-.}/volumes/etcd:/etcd
    command: etcd -advertise-client-urls=http://127.0.0.1:2379 -listen-client-urls http://0.0.0.0:2379 --data-dir /etcd
    healthcheck:
      test: ["CMD", "etcdctl", "endpoint", "health"]
      interval: 30s
      timeout: 20s
      retries: 3

  minio:
    container_name: milvus-minio
    image: minio/minio:RELEASE.2023-03-20T20-16-18Z
    environment:
      MINIO_ACCESS_KEY: minioadmin
      MINIO_SECRET_KEY: minioadmin
    ports:
      - "9091:9001"
      - "9000:9000"
    volumes:
      - ${DOCKER_VOLUME_DIRECTORY:-.}/volumes/minio:/minio_data
    command: minio server /minio_data --console-address ":9001"
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:9000/minio/health/live"]
      interval: 30s
      timeout: 20s
      retries: 3

  standalone:
    container_name: milvus-standalone
    image: milvusdb/milvus:v2.3.3
    command: ["milvus", "run", "standalone"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ${DOCKER_VOLUME_DIRECTORY:-.}/volumes/milvus:/var/lib/milvus
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:9091/healthz"]
      interval: 30s
      start_period: 90s
      timeout: 20s
      retries: 3
    ports:
      - "19530:19530"
      - "9091:9091"
    depends_on:
      - "etcd"
      - "minio"

  attu:
    container_name: attu
    image: zilliz/attu:v2.3.8
    environment:
      MILVUS_URL: milvus-standalone:19530
    ports:
      - "9091:3000"
    depends_on:
      - "standalone"

networks:
  default:
    name: milvus
"""

# Docker Compose 파일 저장
with open("docker-compose.yml", "w") as f:
    f.write(docker_compose_content)

print("✅ Docker Compose 파일이 생성되었습니다.")
print("📍 포트 매핑:")
print("   • Milvus 서버: 19530:19530")
print("   • Milvus 관리 API: 9091:9091")
print("   • Attu 웹 UI: 9091:3000")
print("   • MinIO 콘솔: 9091:9001")
print("   • MinIO API: 9000:9000")
print("\n🚀 서버 실행 명령:")
print("   docker-compose up -d")
print("\n🌐 접속 URL:")
print("   • Attu 관리 도구: http://localhost:9091")
print("   • MinIO 콘솔: http://localhost:9091")

In [ ]:
def create_local_milvus_connection(host="localhost", port=19530):
    """
    로컬 Milvus 서버에 연결하는 함수
    
    Args:
        host (str): Milvus 서버 호스트 (기본값: "localhost")
        port (int): Milvus 서버 포트 (기본값: 19530)
    
    Returns:
        Milvus: Milvus 벡터스토어 또는 None
    
    Usage:
        # Docker Compose로 Milvus 서버 실행 후:
        milvus_db = create_local_milvus_connection()
        if milvus_db:
            print("✅ 연결 성공!")
        else:
            print("❌ 연결 실패")
    """
    try:
        print(f"🔄 로컬 Milvus 서버 연결 시도... ({host}:{port})")
        
        # 로컬 Milvus 서버에 연결
        milvus_db = Milvus.from_documents(
            documents=split_doc1,
            embedding=OpenAIEmbeddings(),
            connection_args={
                "host": host,
                "port": port,
            },
            collection_name="local_documents",
            drop_old=True  # 기존 컬렉션 삭제
        )
        
        print("✅ 로컬 Milvus 서버 벡터 저장소 생성 완료")
        return milvus_db
        
    except Exception as e:
        print(f"❌ Milvus 서버 연결 실패: {e}")
        print(f"💡 해결 방법:")
        print(f"   1. Docker Compose로 서버 실행:")
        print(f"      wget https://github.com/milvus-io/milvus/releases/download/v2.3.3/milvus-standalone-docker-compose.yml -O docker-compose.yml")
        print(f"      docker-compose up -d")
        print(f"   2. 포트 확인: {host}:{port}")
        print(f"   3. Attu 관리 도구: http://localhost:9091")
        return None

# Docker Compose 설정 안내
print("📋 Milvus Docker Compose 설정:")
print("""
# Milvus 공식 Docker Compose 파일 다운로드:
wget https://github.com/milvus-io/milvus/releases/download/v2.3.3/milvus-standalone-docker-compose.yml -O docker-compose.yml

# 포트 매핑 설정 (docker-compose.yml 파일 수정 필요):
# Milvus 서버: 9091:9091, 19530:19530
# Attu 관리 도구: 9091:3000

# 서버 실행:
docker-compose up -d

# 상태 확인:
docker-compose ps

# 관리 도구 Attu 접속:
# http://localhost:9091
""")

# 로컬 서버 연결 시도
local_milvus_db = create_local_milvus_connection()

# 연결 상태 확인
if local_milvus_db is not None:
    print("\n🎯 로컬 Milvus 연결 정보:")
    print(f"   • 서버 주소: localhost:19530")
    print(f"   • 관리 도구: http://localhost:3000 (Attu)")
    print(f"   • 컬렉션명: local_documents")
    print(f"   • 연결 상태: 활성화")
else:
    print("\n⚠️ 로컬 Milvus 서버가 실행되지 않았습니다.")
    print("위의 Docker Compose 명령을 사용하여 서버를 실행하세요.")

📋 Milvus Docker Compose 설정:

# Milvus 공식 Docker Compose 파일 다운로드:
wget https://github.com/milvus-io/milvus/releases/download/v2.3.3/milvus-standalone-docker-compose.yml -O docker-compose.yml

# 포트 매핑 설정 (docker-compose.yml 파일 수정 필요):
# Milvus 서버: 9091:9091, 19530:19530
# Attu 관리 도구: 9091:3000

# 서버 실행:
docker-compose up -d

# 상태 확인:
docker-compose ps

# 관리 도구 Attu 접속:
# http://localhost:9091

🔄 로컬 Milvus 서버 연결 시도... (localhost:19530)
✅ 로컬 Milvus 서버 벡터 저장소 생성 완료

🎯 로컬 Milvus 연결 정보:
   • 서버 주소: localhost:19530
   • 관리 도구: http://localhost:9091 (Attu)
   • 컬렉션명: local_documents
   • 연결 상태: 활성화
✅ 로컬 Milvus 서버 벡터 저장소 생성 완료

🎯 로컬 Milvus 연결 정보:
   • 서버 주소: localhost:19530
   • 관리 도구: http://localhost:9091 (Attu)
   • 컬렉션명: local_documents
   • 연결 상태: 활성화


## 기본 검색

Milvus를 사용한 기본적인 유사도 검색을 수행합니다.

In [10]:
# 사용 가능한 벡터 저장소 확인
milvus_db = None
db_type = None

if milvus_lite_db is not None:
    milvus_db = milvus_lite_db
    db_type = "Milvus Lite"
    print("✅ Milvus Lite 사용")
elif local_milvus_db is not None:
    milvus_db = local_milvus_db
    db_type = "로컬 Milvus"
    print("✅ 로컬 Milvus 사용")
elif zilliz_milvus_db is not None:
    milvus_db = zilliz_milvus_db
    db_type = "Zilliz Cloud"
    print("✅ Zilliz Cloud 사용")
else:
    print("❌ 사용 가능한 Milvus 인스턴스가 없습니다.")

# 벡터 저장소를 사용한 검색
if milvus_db is not None:
    print(f"\n🔍 {db_type} 유사도 검색 수행...")
    
    # 검색 쿼리
    query = "자연어 처리와 머신러닝"
    
    # 유사도 검색
    results = milvus_db.similarity_search(query, k=3)
    
    print(f"\n📝 검색 쿼리: '{query}'")
    print(f"🎯 검색 결과 ({len(results)}개):")
    print("=" * 50)
    
    for i, doc in enumerate(results, 1):
        print(f"\n[{i}] {doc.page_content[:100]}...")
        print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
        
else:
    print("\n⚠️ Milvus에 연결할 수 없어 검색을 수행할 수 없습니다.")

✅ 로컬 Milvus 사용

🔍 로컬 Milvus 유사도 검색 수행...

📝 검색 쿼리: '자연어 처리와 머신러닝'
🎯 검색 결과 (3개):

[1] 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
    Source: data/nlp-keywords.txt

[2] 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정...
    Source: data/nlp-keywords.txt

[3] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

📝 검색 쿼리: '자연어 처리와 머신러닝'
🎯 검색 결과 (3개):

[1] 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
    Source: data/nlp-keywords.txt

[2] 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정...
    Source: data/nlp-keywords.txt

[3] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt


## 점수와 함께 검색

유사도 점수와 함께 검색 결과를 확인합니다.

In [11]:
if milvus_db is not None:
    print(f"🔍 {db_type} 점수와 함께 유사도 검색 수행...")
    
    # 점수와 함께 검색
    query = "딥러닝과 인공지능"
    results_with_scores = milvus_db.similarity_search_with_score(query, k=3)
    
    print(f"\n📝 검색 쿼리: '{query}'")
    print(f"🎯 검색 결과 (점수 포함):")
    print("=" * 60)
    
    for i, (doc, score) in enumerate(results_with_scores, 1):
        print(f"\n[{i}] 유사도 점수: {score:.4f}")
        print(f"    내용: {doc.page_content[:150]}...")
        print(f"    소스: {doc.metadata.get('source', 'Unknown')}")
        
else:
    print("\n⚠️ Milvus에 연결할 수 없어 검색을 수행할 수 없습니다.")

🔍 로컬 Milvus 점수와 함께 유사도 검색 수행...

📝 검색 쿼리: '딥러닝과 인공지능'
🎯 검색 결과 (점수 포함):

[1] 유사도 점수: 0.3359
    내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석...
    소스: data/nlp-keywords.txt

[2] 유사도 점수: 0.3649
    내용: 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정확한 정보를 추출하거나 예측하는 데 사용됩니다.
예시: 이미지와 설명 텍스트를 함께 분석하...
    소스: data/nlp-keywords.txt

[3] 유사도 점수: 0.3736
    내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.
연관키워드: 자...
    소스: data/nlp-keywords.txt

📝 검색 쿼리: '딥러닝과 인공지능'
🎯 검색 결과 (점수 포함):

[1] 유사도 점수: 0.3359
    내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석...
    소스: data/nlp-keywords.txt

[2] 유사도 점수: 0.3649
    내용: 정의: 멀티모달은 여러 종

## 추가 문서 삽입

기존 벡터스토어에 새로운 문서를 추가합니다.

In [12]:
if milvus_db is not None:
    print(f"📄 {db_type}에 추가 문서 삽입...")
    
    # Finance 관련 문서 추가
    milvus_db.add_documents(split_doc2)
    
    print(f"✅ {len(split_doc2)}개의 Finance 문서가 추가되었습니다.")
    
    # 추가된 문서로 검색 테스트
    finance_query = "금융 시장과 투자"
    finance_results = milvus_db.similarity_search(finance_query, k=3)
    
    print(f"\n🔍 Finance 관련 검색: '{finance_query}'")
    print("=" * 50)
    
    for i, doc in enumerate(finance_results, 1):
        print(f"\n[{i}] {doc.page_content[:100]}...")
        print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
        
else:
    print("\n⚠️ Milvus에 연결할 수 없어 문서 추가를 수행할 수 없습니다.")

📄 로컬 Milvus에 추가 문서 삽입...
✅ 6개의 Finance 문서가 추가되었습니다.
✅ 6개의 Finance 문서가 추가되었습니다.

🔍 Finance 관련 검색: '금융 시장과 투자'

[1] 정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.
예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별...
    Source: data/finance-keywords.txt

[2] 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.
연관키워...
    Source: data/finance-keywords.txt

[3] 정의: 블루칩 주식은 재무적으로 안정적이고 오랜 기간 동안 꾸준한 실적을 보여온 대형 기업의 주식을 의미합니다.
예시: 존슨앤존슨, 프록터앤갬블과 같은 기업들은 S&P 500에 포...
    Source: data/finance-keywords.txt

🔍 Finance 관련 검색: '금융 시장과 투자'

[1] 정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.
예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별...
    Source: data/finance-keywords.txt

[2] 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.
연관키워...
    Source: data/finance-keywords.txt

[3] 정의: 블루칩 주식은 재무적으로 안정적이고 오랜 기간 동안 꾸준한 실적을 보여온 대형 기업의 주식을 의미합니다.
예시: 존슨앤존슨, 프록터앤갬블과 같은 기업들은 S&P 500에 포...
    Source: data/

## 고급 검색 옵션

Milvus의 고급 검색 옵션을 사용합니다.

In [13]:
if milvus_db is not None:
    print(f"🔍 {db_type} 고급 검색 옵션...")
    
    try:
        # 메타데이터 필터링을 포함한 검색
        query = "분석과 데이터"
        
        # 검색 매개변수 설정
        search_kwargs = {
            "k": 5,
            "param": {"nprobe": 16},  # 검색 정확도 조정
        }
        
        # 고급 검색 수행
        advanced_results = milvus_db.similarity_search(
            query, 
            k=search_kwargs["k"]
        )
        
        print(f"\n📝 고급 검색 쿼리: '{query}'")
        print(f"🎯 검색 결과 ({len(advanced_results)}개):")
        print("=" * 50)
        
        for i, doc in enumerate(advanced_results, 1):
            print(f"\n[{i}] {doc.page_content[:120]}...")
            print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
            
    except Exception as e:
        print(f"❌ 고급 검색 실행 중 오류: {e}")
        print("💡 기본 검색으로 대체합니다.")
        
        # 기본 검색으로 대체
        basic_results = milvus_db.similarity_search(query, k=3)
        
        for i, doc in enumerate(basic_results, 1):
            print(f"\n[{i}] {doc.page_content[:120]}...")
            print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
        
else:
    print("\n⚠️ Milvus에 연결할 수 없어 고급 검색을 수행할 수 없습니다.")

🔍 로컬 Milvus 고급 검색 옵션...

📝 고급 검색 쿼리: '분석과 데이터'
🎯 검색 결과 (5개):

[1] 정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "pro...
    Source: data/nlp-keywords.txt

[2] 정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 ...
    Source: data/nlp-keywords.txt

[3] 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정확한 정보를 추출하거나 예측하는 데 ...
    Source: data/nlp-keywords.txt

[4] 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: 많은 문서에서 자주 등장하지 않는 단...
    Source: data/nlp-keywords.txt

[5] 정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이...
    Source: data/nlp-keywords.txt

📝 고급 검색 쿼리: '분석과 데이터'
🎯 검색 결과 (5개):

[1] 정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love p

## 컬렉션 정보 확인

Milvus 컬렉션의 정보를 확인합니다.

In [14]:
if milvus_db is not None:
    print(f"📋 {db_type} 컬렉션 정보...")
    
    try:
        # Milvus 컬렉션 객체 접근
        collection = milvus_db.col
        
        print("\n🏗️ 컬렉션 정보:")
        print("=" * 40)
        
        # 기본 정보
        print(f"   컬렉션명: {collection.name}")
        print(f"   설명: {collection.description}")
        
        # 스키마 정보
        schema = collection.schema
        print(f"   필드 수: {len(schema.fields)}")
        
        print("\n📝 필드 정보:")
        for field in schema.fields:
            field_name = field.name
            field_type = field.dtype.name if hasattr(field.dtype, 'name') else str(field.dtype)
            is_primary = field.is_primary
            print(f"   - {field_name}: {field_type}" + (" (Primary)" if is_primary else ""))
        
        # 통계 정보
        collection.load()  # 컬렉션 로드
        num_entities = collection.num_entities
        print(f"\n📊 데이터 통계:")
        print(f"   총 엔티티 수: {num_entities}")
        
        # 인덱스 정보
        indexes = collection.indexes
        print(f"   인덱스 수: {len(indexes)}")
        
        if indexes:
            for index in indexes:
                print(f"   - 필드: {index.field_name}")
                print(f"     타입: {index.params.get('index_type', 'Unknown')}")
                print(f"     메트릭: {index.params.get('metric_type', 'Unknown')}")
                
    except Exception as e:
        print(f"❌ 컬렉션 정보 조회 중 오류: {e}")
        print("💡 컬렉션이 완전히 초기화되지 않았을 수 있습니다.")
        
else:
    print("\n⚠️ Milvus에 연결할 수 없어 컬렉션 정보를 확인할 수 없습니다.")

📋 로컬 Milvus 컬렉션 정보...

🏗️ 컬렉션 정보:
   컬렉션명: local_documents
   설명: 
   필드 수: 4

📝 필드 정보:
   - text: VARCHAR
   - pk: INT64 (Primary)
   - vector: FLOAT_VECTOR
   - source: VARCHAR

📊 데이터 통계:
   총 엔티티 수: 0
   인덱스 수: 1
   - 필드: vector
     타입: AUTOINDEX
     메트릭: L2


## Retriever로 사용

Milvus VectorStore를 LangChain Retriever로 사용합니다.

In [15]:
if milvus_db is not None:
    print(f"🔄 {db_type} Retriever로 사용...")
    
    # Retriever 생성
    retriever = milvus_db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}
    )
    
    # Retriever 사용
    query = "데이터 분석과 시각화"
    retrieved_docs = retriever.invoke(query)
    
    print(f"\n📝 Retriever 쿼리: '{query}'")
    print(f"🎯 검색된 문서 ({len(retrieved_docs)}개):")
    print("=" * 50)
    
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\n[{i}] {doc.page_content[:120]}...")
        print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
        
else:
    print("\n⚠️ Milvus 벡터스토어가 사용 불가능합니다.")

🔄 로컬 Milvus Retriever로 사용...


I0000 00:00:1760687042.895227  200955 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')



📝 Retriever 쿼리: '데이터 분석과 시각화'
🎯 검색된 문서 (4개):

[1] 정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이...
    Source: data/nlp-keywords.txt

[2] 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: 많은 문서에서 자주 등장하지 않는 단...
    Source: data/nlp-keywords.txt

[3] 정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "pro...
    Source: data/nlp-keywords.txt

[4] 정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 ...
    Source: data/nlp-keywords.txt


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')


## 성능 테스트

Milvus의 검색 성능을 측정합니다.

In [16]:
if milvus_db is not None:
    import time
    
    print(f"⚡ {db_type} 성능 테스트...")
    
    # 테스트 쿼리들
    test_queries = [
        "자연어 처리",
        "머신러닝 알고리즘",
        "딥러닝 모델",
        "데이터 분석",
        "인공지능 응용"
    ]
    
    total_time = 0
    total_queries = len(test_queries)
    
    print(f"\n🔍 {total_queries}개 쿼리 성능 측정:")
    print("=" * 50)
    
    for i, query in enumerate(test_queries, 1):
        start_time = time.time()
        
        # 검색 수행
        results = milvus_db.similarity_search(query, k=3)
        
        end_time = time.time()
        query_time = end_time - start_time
        total_time += query_time
        
        print(f"[{i}] '{query}': {query_time:.4f}초 ({len(results)}개 결과)")
    
    # 성능 요약
    avg_time = total_time / total_queries
    qps = total_queries / total_time
    
    print(f"\n📊 성능 요약:")
    print(f"   총 시간: {total_time:.4f}초")
    print(f"   평균 응답 시간: {avg_time:.4f}초")
    print(f"   초당 쿼리 수 (QPS): {qps:.2f}")
    print(f"   데이터베이스: {db_type}")
        
else:
    print("\n⚠️ Milvus 벡터스토어가 사용 불가능하여 성능 테스트를 수행할 수 없습니다.")

⚡ 로컬 Milvus 성능 테스트...

🔍 5개 쿼리 성능 측정:
[1] '자연어 처리': 0.7723초 (3개 결과)
[1] '자연어 처리': 0.7723초 (3개 결과)
[2] '머신러닝 알고리즘': 0.3185초 (3개 결과)
[2] '머신러닝 알고리즘': 0.3185초 (3개 결과)
[3] '딥러닝 모델': 0.2801초 (3개 결과)
[3] '딥러닝 모델': 0.2801초 (3개 결과)
[4] '데이터 분석': 0.3176초 (3개 결과)
[4] '데이터 분석': 0.3176초 (3개 결과)
[5] '인공지능 응용': 0.2788초 (3개 결과)

📊 성능 요약:
   총 시간: 1.9672초
   평균 응답 시간: 0.3934초
   초당 쿼리 수 (QPS): 2.54
   데이터베이스: 로컬 Milvus
[5] '인공지능 응용': 0.2788초 (3개 결과)

📊 성능 요약:
   총 시간: 1.9672초
   평균 응답 시간: 0.3934초
   초당 쿼리 수 (QPS): 2.54
   데이터베이스: 로컬 Milvus
